# `anato-mesh` in Jupyter Notebook 

`anato_mesh.py` contains the main functions for calculating the partition-level curvatures using python to reproduce the algorithm originally published by K. Khabaz here: https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1011815

In [ ]:
from anato_mesh import *
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [ ]:
#- Pocivavsek lab parent directory
parent_path = 'Z:\\aorta\\aortas'
#- Cohort groups filter
group_str = ['KK', 'KY']
#- Mesh formats filter
file_str = ['M5', 'M10']

| `quantities` | `point_removal` |
| --- | --- |
| 'Gaussian' | 'curvature' | 
| 'Casorati' | 'thoracic' | 
| 'IntMean' |
| 'Total' |
| 'ShapeIndex' |
| 'Willmore' |

In [ ]:
quantities = ['Casorati','Total']
point_removal = 'thoracic'

The number of surface partitions for each surface is determined by the following equation. `m_set` provides an easily adjustable parameter for further partition scaling and allows you to calculate more than one scaling at once. The optimized value for the thoracic aorta TEVAR dataset is `m=1`.

$$partitions = m \times \left(\frac{SA}{R^2}\right)$$

In [ ]:
m_set = [0.5, 1, 5]

In [ ]:
results = GetAortaMeshResults(parent_path, group_str, file_str, point_removal, quantities, m_set)

In [ ]:
results

In [ ]:
#- Meta data integration 
directory = 'Z:\\aorta\\meta_data'
file_name = 'MasterMetaData.xlsx'
#- Cohort meta data group names
cohort_list = ['UC_NORMAL', 'UC_TEVAR']
#- Categorical labels 
cat_columns = ['Outcome', 'Label', 'Chronicity']

In [ ]:
results_norm = GroupsNormalize(SplitUpScanName(results), 'Casorati_Mean', 'Total_Fluct', 'KY')

In [ ]:
results_meta = MergeMetaData(directory, file_name, cohort_list, results_norm, cat_columns)

In [ ]:
SaveToXLSX('Z:\\aorta\\projects\\chronicity', 'Chron_Fall24.xlsx', results_meta)